In [ ]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
device='cuda'
model.bfloat16() # keep if you have less than 8G GPU memory
model.to(device)

In [52]:
# difflib less accurate but included in standard python library
SIMILARITY_METHOD = "tf-idf" # requires scikit-learn
# SIMILARITY_METHOD = "difflib"

In [53]:
import csv
# load Q&A db based on russian Jeopardy dataset
db = dict()
with open("db2.tsv") as fd:
    reader = csv.reader(fd, delimiter="\t")
    next(reader) # skip header
    for row in reader:
        db[row[0]] = row[1]

In [54]:
queries = '''Плоды какого дерева семейства цитрусовых висят как гроздья?
Какой метод лечения основал Ганеман?
Что такое борт номер один?
Как нужно отпускать сцепление?
Кто был третий чемпион мира по шахматам?
'''.strip().split("\n")

In [55]:
# find top 10 similar questions with difflib or tf-idf
if SIMILARITY_METHOD == 'difflib':
    import difflib
    def context(query, n=10):
        return '\n'.join(['Пользователь: %s\nАссистент: %s'% (k, db[k]) for k in difflib.get_close_matches(query, db.keys(), n=n, cutoff=0.1)])

In [56]:
if SIMILARITY_METHOD == 'tf-idf':
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.neighbors import NearestNeighbors
    vectorizer = TfidfVectorizer()
    nn = NearestNeighbors(metric='cosine')
    nn.fit(vectorizer.fit_transform(db.keys()))
    
    def context(query, n=10):
        distances, indices = nn.kneighbors(vectorizer.transform([query]), n_neighbors = n)
        lst = list(db.keys())
        return '\n'.join(['Пользователь: %s\nАссистент: %s'% (lst[i], db[lst[i]]) for i in indices[0]])


In [57]:
def t5(query, context=""):
    lm_text = """<SC6>Продолжи диалог:
Пользователь: Привет. Ты кто?
Ассистент: Я умный персональный ассистент. Отвечаю на различные вопросы пользователя.
%s
Пользователь: %s
Ассистент: <extra_id_0>
Пользователь: Спасибо, ты очень помог. Пока!
Ассистент: Всегда рад помочь.
""" % (context, query)
    input_ids = torch.tensor([tokenizer.encode(lm_text)]).to(device)
    outputs = model.generate(input_ids, eos_token_id=tokenizer.eos_token_id,
                         num_beams=4,
                         repetition_penalty=1.04,
                         temperature=0.3,
                         max_length=150,
                         min_length=1
                         )
    t5_output = tokenizer.decode(outputs[0][1:])

    if '</s>' in t5_output:
        t5_output = t5_output[:t5_output.find('</s>')].strip()
    t5_output = t5_output.replace('<extra_id_0>', '').strip()
    t5_output = t5_output.replace('Ассистент:', '').strip()
    t5_output = t5_output.split('Пользователь:')[0].strip()
    return t5_output
for query in queries:
    print("Question:")
    print("\t", query)
    print("Without context:")
    print("\t", t5(query))
    print("With context:")
    с = context(query, 5)
    # print(с)
    print("\t", t5(query, с))

Question:
	 Плоды какого дерева семейства цитрусовых висят как гроздья?
Without context:
	 Лимонное дерево.
With context:
	 Грейпфрут
Question:
	 Какой метод лечения основал Ганеман?
Without context:
	 Метод Ганемана основан на том, что все болезни от нервов.
With context:
	 Гомеопатия
Question:
	 Что такое борт номер один?
Without context:
	 Борт номер один - это самолет, который летит первым.
With context:
	 Президентский самолет
Question:
	 Как нужно отпускать сцепление?
Without context:
	 Сцепление нужно отпускать плавно.
With context:
	 Ме-е-е-дленно
Question:
	 Кто был третий чемпион мира по шахматам?
Without context:
	 Гарри Каспаров.
With context:
	 АЛЕКСАНДР АЛЬБЕРТОВИЧ
